## Advanced Example - Movie Recommendation System

In [1]:
from pydantic import BaseModel
from typing import List, Optional, Dict, Any
from pydapter.extras.neo4j_ import Neo4jAdapter
from neo4j import GraphDatabase
import random

# Neo4j connection settings
NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j", "password")


# Define our models
class Person(BaseModel):
    id: str
    name: str
    age: Optional[int] = None


class Movie(BaseModel):
    id: str
    title: str
    year: int
    genre: List[str] = []
    rating: Optional[float] = None


class Actor(Person):
    roles: List[str] = []


class Director(Person):
    movies_directed: int = 0


# Helper function to create Neo4j driver
def get_driver():
    return GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)


# Initialize the database with schema and constraints
def initialize_database():
    driver = get_driver()

    with driver.session() as session:
        # Create constraints to ensure uniqueness
        session.run(
            "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Person) REQUIRE p.id IS UNIQUE"
        )
        session.run(
            "CREATE CONSTRAINT IF NOT EXISTS FOR (m:Movie) REQUIRE m.id IS UNIQUE"
        )

    driver.close()
    print("Database initialized with constraints")


# Helper function to create relationships
def create_relationship(
    start_id, end_id, start_label, end_label, rel_type, properties=None
):
    driver = get_driver()

    props_str = ""
    if properties:
        props_list = [f"{k}: ${k}" for k in properties.keys()]
        props_str = "{" + ", ".join(props_list) + "}"

    with driver.session() as session:
        query = f"""
        MATCH (a:{start_label} {{id: $start_id}})
        MATCH (b:{end_label} {{id: $end_id}})
        MERGE (a)-[r:{rel_type} {props_str}]->(b)
        RETURN a.name, b.title
        """

        params = {"start_id": start_id, "end_id": end_id}
        if properties:
            params.update(properties)

        result = session.run(query, params)
        data = result.single()
        if data:
            print(f"Created relationship: {data[0]} {rel_type} {data[1]}")

    driver.close()


# Populate the database with sample data
def populate_database():
    # Create some movies
    movies = [
        Movie(
            id="m1",
            title="The Matrix",
            year=1999,
            genre=["Sci-Fi", "Action"],
            rating=8.7,
        ),
        Movie(
            id="m2",
            title="Inception",
            year=2010,
            genre=["Sci-Fi", "Action", "Thriller"],
            rating=8.8,
        ),
        Movie(
            id="m3",
            title="The Shawshank Redemption",
            year=1994,
            genre=["Drama"],
            rating=9.3,
        ),
        Movie(
            id="m4",
            title="Pulp Fiction",
            year=1994,
            genre=["Crime", "Drama"],
            rating=8.9,
        ),
        Movie(
            id="m5",
            title="The Dark Knight",
            year=2008,
            genre=["Action", "Crime", "Drama"],
            rating=9.0,
        ),
    ]

    # Create some actors
    actors = [
        Actor(id="a1", name="Keanu Reeves", age=57, roles=["Neo", "John Wick"]),
        Actor(
            id="a2", name="Leonardo DiCaprio", age=46, roles=["Dom Cobb", "Jack Dawson"]
        ),
        Actor(
            id="a3", name="Morgan Freeman", age=84, roles=["Ellis Boyd 'Red' Redding"]
        ),
        Actor(id="a4", name="Tim Robbins", age=62, roles=["Andy Dufresne"]),
        Actor(id="a5", name="John Travolta", age=67, roles=["Vincent Vega"]),
        Actor(id="a6", name="Samuel L. Jackson", age=72, roles=["Jules Winnfield"]),
        Actor(id="a7", name="Christian Bale", age=47, roles=["Bruce Wayne"]),
    ]

    # Create some directors
    directors = [
        Director(id="d1", name="Lana Wachowski", age=56, movies_directed=5),
        Director(id="d2", name="Christopher Nolan", age=51, movies_directed=11),
        Director(id="d3", name="Frank Darabont", age=62, movies_directed=4),
        Director(id="d4", name="Quentin Tarantino", age=58, movies_directed=9),
    ]

    # Store movies in Neo4j
    print("Storing movies...")
    for movie in movies:
        Neo4jAdapter.to_obj(
            movie, url=NEO4J_URI, auth=NEO4J_AUTH, label="Movie", merge_on="id"
        )

    # Store actors in Neo4j
    print("\nStoring actors...")
    for actor in actors:
        # Store using Neo4jAdapter
        Neo4jAdapter.to_obj(
            actor,
            url=NEO4J_URI,
            auth=NEO4J_AUTH,
            label="Actor",  # Use Actor label
            merge_on="id",
        )

    # Store directors in Neo4j
    print("\nStoring directors...")
    for director in directors:
        Neo4jAdapter.to_obj(
            director,
            url=NEO4J_URI,
            auth=NEO4J_AUTH,
            label="Director",  # Use Director label
            merge_on="id",
        )

    # Create relationships
    print("\nCreating relationships...")

    # Matrix relationships
    create_relationship("a1", "m1", "Actor", "Movie", "ACTED_IN", {"role": "Neo"})
    create_relationship("d1", "m1", "Director", "Movie", "DIRECTED")

    # Inception relationships
    create_relationship("a2", "m2", "Actor", "Movie", "ACTED_IN", {"role": "Dom Cobb"})
    create_relationship("d2", "m2", "Director", "Movie", "DIRECTED")

    # Shawshank Redemption relationships
    create_relationship(
        "a3", "m3", "Actor", "Movie", "ACTED_IN", {"role": "Ellis Boyd 'Red' Redding"}
    )
    create_relationship(
        "a4", "m3", "Actor", "Movie", "ACTED_IN", {"role": "Andy Dufresne"}
    )
    create_relationship("d3", "m3", "Director", "Movie", "DIRECTED")

    # Pulp Fiction relationships
    create_relationship(
        "a5", "m4", "Actor", "Movie", "ACTED_IN", {"role": "Vincent Vega"}
    )
    create_relationship(
        "a6", "m4", "Actor", "Movie", "ACTED_IN", {"role": "Jules Winnfield"}
    )
    create_relationship("d4", "m4", "Director", "Movie", "DIRECTED")

    # Dark Knight relationships
    create_relationship(
        "a7", "m5", "Actor", "Movie", "ACTED_IN", {"role": "Bruce Wayne"}
    )
    create_relationship("d2", "m5", "Director", "Movie", "DIRECTED")

    # Create user ratings
    create_user_ratings()

    print("Database populated with sample data")


# Create some users and their ratings
def create_user_ratings():
    # Create users
    users = [
        Person(id="u1", name="User One", age=25),
        Person(id="u2", name="User Two", age=35),
        Person(id="u3", name="User Three", age=45),
    ]

    # Store users
    print("\nStoring users...")
    for user in users:
        Neo4jAdapter.to_obj(
            user, url=NEO4J_URI, auth=NEO4J_AUTH, label="User", merge_on="id"
        )

    # Create rating relationships
    driver = get_driver()

    with driver.session() as session:
        # Get all movie IDs
        result = session.run("MATCH (m:Movie) RETURN m.id AS id")
        movie_ids = [record["id"] for record in result]

        # For each user, create some random ratings
        for user_id in ["u1", "u2", "u3"]:
            for movie_id in movie_ids:
                # Randomly decide if user rated this movie
                if random.random() > 0.3:  # 70% chance of rating
                    rating = (
                        round(random.uniform(1, 5) * 2) / 2
                    )  # Rating from 1 to 5, in 0.5 steps

                    session.run(
                        """
                        MATCH (u:User {id: $user_id})
                        MATCH (m:Movie {id: $movie_id})
                        MERGE (u)-[r:RATED]->(m)
                        SET r.rating = $rating
                        """,
                        user_id=user_id,
                        movie_id=movie_id,
                        rating=rating,
                    )
                    print(f"User {user_id} rated movie {movie_id} with {rating}")

    driver.close()


# Function to get movie recommendations for a user
def get_movie_recommendations(user_id):
    """
    Get movie recommendations for a user based on:
    1. Movies they haven't seen
    2. Movies liked by users with similar tastes
    3. Movies in genres they like
    """
    driver = get_driver()

    recommendations = []

    with driver.session() as session:
        # Get movies the user hasn't rated,
        # but are highly rated by users with similar tastes
        result = session.run(
            """
            MATCH (target:User {id: $user_id})-[r1:RATED]->(m:Movie)
            MATCH (other:User)-[r2:RATED]->(m)
            WHERE other.id <> $user_id AND abs(r1.rating - r2.rating) < 1
            MATCH (other)-[r3:RATED]->(rec:Movie)
            WHERE r3.rating >= 4
            AND NOT EXISTS { MATCH (target)-[:RATED]->(rec) }
            WITH rec, count(*) AS strength, avg(r3.rating) AS avg_rating
            ORDER BY strength DESC, avg_rating DESC
            LIMIT 5
            RETURN rec
            """,
            user_id=user_id,
        )

        for record in result:
            movie_data = dict(record["rec"].items())
            movie = Movie(**movie_data)
            recommendations.append(movie)

    driver.close()
    return recommendations


# Get movies directed by a specific director
def get_movies_by_director(director_name):
    """Get all movies directed by a specific director"""
    driver = get_driver()

    movies_list = []

    with driver.session() as session:
        result = session.run(
            """
            MATCH (d:Director {name: $director_name})-[:DIRECTED]->(m:Movie)
            RETURN m
            """,
            director_name=director_name,
        )

        for record in result:
            movie_data = dict(record["m"].items())
            movie = Movie(**movie_data)
            movies_list.append(movie)

    driver.close()
    return movies_list


# Get actors who worked with a specific actor
def get_co_actors(actor_name):
    """Get all actors who acted in the same movie as the specified actor"""
    driver = get_driver()

    co_actors = []

    with driver.session() as session:
        result = session.run(
            """
            MATCH (a:Actor {name: $actor_name})-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(co:Actor)
            WHERE co.name <> $actor_name
            RETURN DISTINCT co
            """,
            actor_name=actor_name,
        )

        for record in result:
            actor_data = dict(record["co"].items())
            actor = Actor(**actor_data)
            co_actors.append(actor)

    driver.close()
    return co_actors


# Main function to demo the movie recommendation system
def main():
    # Initialize and populate the database
    initialize_database()
    populate_database()

    # Get movie recommendations for User One
    print("\nMovie recommendations for User One:")
    recommendations = get_movie_recommendations("u1")
    for movie in recommendations:
        print(f"  - {movie.title} ({movie.year}) - Rating: {movie.rating}")

    # Get movies directed by Christopher Nolan
    print("\nMovies directed by Christopher Nolan:")
    nolan_movies = get_movies_by_director("Christopher Nolan")
    for movie in nolan_movies:
        print(f"  - {movie.title} ({movie.year}) - Rating: {movie.rating}")

    # Get actors who worked with Keanu Reeves
    print("\nActors who worked with Keanu Reeves:")
    keanu_co_actors = get_co_actors("Keanu Reeves")
    for actor in keanu_co_actors:
        print(f"  - {actor.name} (Age: {actor.age})")


main()

Database initialized with constraints
Storing movies...

Storing actors...

Storing directors...

Creating relationships...
Created relationship: Keanu Reeves ACTED_IN The Matrix
Created relationship: Lana Wachowski DIRECTED The Matrix
Created relationship: Leonardo DiCaprio ACTED_IN Inception
Created relationship: Christopher Nolan DIRECTED Inception
Created relationship: Morgan Freeman ACTED_IN The Shawshank Redemption
Created relationship: Tim Robbins ACTED_IN The Shawshank Redemption
Created relationship: Frank Darabont DIRECTED The Shawshank Redemption
Created relationship: John Travolta ACTED_IN Pulp Fiction
Created relationship: Samuel L. Jackson ACTED_IN Pulp Fiction
Created relationship: Quentin Tarantino DIRECTED Pulp Fiction
Created relationship: Christian Bale ACTED_IN The Dark Knight
Created relationship: Christopher Nolan DIRECTED The Dark Knight

Storing users...
User u1 rated movie m1 with 4.5
User u1 rated movie m3 with 1.5
User u1 rated movie m4 with 3.5
User u1 rated